In [90]:
import sys
import psutil
import numpy as np
import pandas as pd
import plotly.io as pio
from collections import Counter
import dataframe_image as dfi
import plotly.graph_objects as go


In [91]:
# !{sys.executable} -m pip install  -U imgkit


# debug accounts 

In [92]:
# f = pd.read_csv('bbcm-china-source-data_fb-accounts-v3-ct-imported-pages.csv')
# x= f['crowdtangle.account/id']
# l = pd.read_excel('BBCM - Masterfile.xlsx', engine='openpyxl')
# l = l.iloc[:,0:22].fillna('None')
# l = l[l['M52/platform']=='Facebook']
# l[l['M52/accountId']=='crowdtangle-3701035']
# y = l['M52/accountId'].str.replace('crowdtangle-','')
# y = y[:196] 
# b = [int(i) for i in y ]
# c = [int(j) for j in x ]
# for i, j in zip(sorted(b),sorted(c)):
#         print(i,j)


In [109]:
fb_collectable = pd.read_csv('data_collection/fb-accounts-v3-ct-imported-pages.csv')
tw_collectable = pd.read_csv('data_collection/tw-accounts-v3-collectable.csv')
fb_accounts_collections = pd.read_csv('data_collection/fb-v3-post-collections-20210402.csv')
tw_accounts_collections = pd.read_csv('data_collection/tw-v3-tweet-collections.csv')
consolidated_accounts_collection = pd.read_csv('data_collection/store-all-messages-v3.csv')

# Acoounts 

In [122]:
class accounts_split:
    
    def __init__(self, version):
        self.version = version
        self.langs = ['ar', 'en', 'fr', 'es']
        self.master_file = pd.read_excel(f'{self.version}/data_collection/BBCM - Masterfile.xlsx', engine='openpyxl')
        self.fb_collectable = pd.read_csv(f'{self.version}/data_collection/fb-accounts-v3-ct-imported-pages.csv')
        self.tw_collectable = pd.read_csv(f'{self.version}/data_collection/tw-accounts-v3-collectable.csv')
        self.fb_accounts_collections = pd.read_csv(f'{self.version}/data_collection/fb-v3-post-collections-20210402.csv')
        self.tw_accounts_collections = pd.read_csv(f'{self.version}/data_collection/tw-v3-tweet-collections.csv')
        self.consolidated_accounts_collection = pd.read_csv(f'{self.version}/data_collection/store-all-messages-v3.csv')
        self.acc_sum = self.accounts_summary(self.master_file, self.fb_collectable, self.tw_collectable, self.tw_accounts_collections, self.fb_accounts_collections, self.consolidated_accounts_collection)
        
    def annotate(self, folder):
        self.fig = self.accounts_annotation(self.acc_sum)
        pio.write_image(self.fig, f"{self.version}/{folder}/accounts_collection_split_v3.jpeg")
    
    def create_dict(self, lst):
        res_dct = {}
        for i,t in enumerate(lst):
            res_dct[t] = i
        return res_dct
        


    def accounts_summary(self, master, tw, fb, tw_coll, fb_coll, lang_coll):
        all_accounts = len(master.iloc[:,:22].iloc[:563])
        fb_accounts = len(fb)
        tw_accounts = len(tw)
        tw_collections = len(tw_coll)
        fb_collections = len(fb_coll)
        collectable = fb_accounts + tw_accounts
        not_collectable = all_accounts - collectable
        self.acc_coll_sum = self.accounts_collection_summary(lang_coll)

        return [all_accounts, fb_accounts, tw_accounts, collectable, not_collectable, tw_collections, fb_collections,  tw_collections + fb_collections] + self.acc_coll_sum

    def accounts_collection_summary(self, df):
        lang_collection = []
        for lang in self.langs:
            lang_collection.append(len(df[df['M52/language'] == lang]))
        lang_collection.append(np.sum(lang_collection))
        print(lang_collection)
        return lang_collection
    
    def account_prefixes(self, acc_sum, account_sources):
        annotated = []
        filtered_collections = self.account_collection_prefixes(self.acc_coll_sum, account_sources[-4:])
        annotated.append(f'{account_sources[0]} - {acc_sum[0]}')
        annotated.append(f'{account_sources[1]} - {acc_sum[3]}')
        annotated.append(f'{account_sources[2]} - {acc_sum[4] - 2}')
        annotated.append(f'{account_sources[3]} - {acc_sum[1]} ')
        annotated.append(f'{account_sources[4]} - {acc_sum[2]} ')
        annotated.append(f'{account_sources[5]} - {acc_sum[5]} ')
        annotated.append(f'{account_sources[6]} - {acc_sum[6]} ')
        annotated.append(f'{account_sources[7]} - {acc_sum[7]} ')
        return annotated + filtered_collections


    def account_collection_prefixes(self, acc_coll_sum, sources):
        annotated = []
        annotated.append(f'{sources[0]} - {acc_coll_sum[0]}')
        annotated.append(f'{sources[1]} - {acc_coll_sum[1]}')
        annotated.append(f'{sources[2]} - {acc_coll_sum[2]}')
        annotated.append(f'{sources[3]} - {acc_coll_sum[3]}')

        return annotated 
    
    def accounts_annotation(self, acc_sum):
        account_sources = ["source_accounts", "collectable", "not collectable", "Twitter accounts", "Facebook accounts", "Twitter collection", "Facebook collection", "Consolidate", "Arabic", "English", "Spanish", "French"]
        account_sources_map = self.create_dict(account_sources)
        account_labels = self.account_prefixes(acc_sum, account_sources)
        fig = go.Figure(data=[go.Sankey(
            arrangement = "snap",
            valueformat = ".0f",
            node = dict(
              pad = 15,
              thickness = 20,
              line = dict(color = "black", width = 0.5),
              label = account_labels,
              x =  [0, 0.22, 0.1, 0,  0,  0.61, 0.5, 0.84, 1, 1, 1, 1 ],
              y =  [0, 0.1,   1,  0, 0.9,  0.1, 0.7, 0.3, 0.6, 0.7, 0.8, 1 ],
            ),
            link = dict(
              source = [account_sources_map[account_sources[0]], account_sources_map[account_sources[0]], account_sources_map[account_sources[1]], account_sources_map[account_sources[1]], account_sources_map[account_sources[3]], account_sources_map[account_sources[4]], account_sources_map[account_sources[5]], account_sources_map[account_sources[6]], account_sources_map[account_sources[7]], account_sources_map[account_sources[7]], account_sources_map[account_sources[7]],  account_sources_map[account_sources[7]]  ], 
              target = [account_sources_map[account_sources[1]], account_sources_map[account_sources[2]], account_sources_map[account_sources[3]], account_sources_map[account_sources[4]], account_sources_map[account_sources[5]], account_sources_map[account_sources[6]], account_sources_map[account_sources[7]], account_sources_map[account_sources[7]], account_sources_map[account_sources[8]], account_sources_map[account_sources[9]], account_sources_map[account_sources[10]], account_sources_map[account_sources[11]]  ],
              value =  [acc_sum[0]/acc_sum[0],                   acc_sum[4]/acc_sum[0],                   acc_sum[2]/acc_sum[0],                   acc_sum[1]/acc_sum[0],                   acc_sum[5]/acc_sum[7],                   acc_sum[6]/acc_sum[7],                   acc_sum[5]/acc_sum[7],                   acc_sum[6]/acc_sum[7],                   acc_sum[8]/acc_sum[12],                  acc_sum[9]/acc_sum[12],                  acc_sum[10]/acc_sum[12],                  acc_sum[11]/acc_sum[12]   ]
          ))])

        fig.update_layout(title_text="New accounts collection", font_size=12)
        fig.show()
        return fig


# Keyword annotators

In [123]:
class keyword_annotator_split:
    
    def __init__(self, version):
        self.version = version
        self.langs = ['ar', 'en', 'fr', 'es']
        self.lang_map = {'ar': 'Arabic', 'en': 'English', 'es': 'Spanish', 'fr': 'French'}
        self.language_data = pd.read_csv(f'{self.version}/keyword_annotations/store-all-messages.csv')
        self.lang_sum = self.language_summary(self.language_data)
        
    def annotate(self, folder):
        langs_analysis = []
        for lang in self.langs:
            annotated_data = pd.read_csv(f'{self.version}/keyword_annotations/store-all-messages-annotated-v1-{lang}.csv')
            self.themes = [t for t in list(annotated_data.columns) if t !='M52/language']
            self.themes_mapping = dict(zip(list(self.themes),['politics and society', 'technology', 'covid-19', 'geopolitics', 'environment', 'military and security', 'economy', 'chinese culture and people']))
            annotated_renamed = self.rename_columns(annotated_data)
            themes_count = self.annotation_count(annotated_renamed)   
            self.fig = self.keyword_annotation(self.lang_sum['all'], lang, self.lang_sum[lang], themes_count)
            pio.write_image(self.fig, f"{self.version}/{folder}/keyword_annotator_split_v2_{lang}.png")
        
            
    def language_summary(self, language_data):
        langs_proportions = {}
        for lang in self.langs:
            langs_proportions[lang] = len(language_data[language_data['M52/language'] == lang])
        langs_proportions['all'] = len(language_data)
        return langs_proportions
        

    def rename_columns(self, df):
        renamed = df.rename(columns = self.themes_mapping)
        return renamed

    def annotation_count(self, df):
        theme_count = {}
        for t in list(self.themes_mapping.values()):
            theme_count[t] = df[t].value_counts()[1]
        return theme_count

    def create_dict(self, lst):
        res_dct = {}
        for i,t in enumerate(lst):
            res_dct[t] = i
        return res_dct

    def source_prefixes(self, all_proportion, proportion, sources, annotation_count):
        annotated = []
        annotated.append(f'{sources[0]} - {proportion}')
        for i, ant in enumerate(sources[1:-1]):
            annotated.append(f'{ant} - {annotation_count[sources[i+1]]}')
        return annotated + [f'{sources[-1:][0]} - {all_proportion}']

    def keyword_annotation(self, all_proportion, lang, porportion, annotation_count):
        sources = [ lang, "economy", "politics and society", "covid-19", "technology", "environment", "military and security", "geopolitics", "chinese culture and people", 'source data']    
        sources_map = self.create_dict(sources)
        labels = self.source_prefixes(all_proportion,  porportion, sources, annotation_count)
        fig = go.Figure(data=[go.Sankey(
            arrangement = "snap",
             valueformat = ".0f",
            node = dict(
              pad = 15,
              thickness = 20,
              line = dict(color = "black", width = 0.5),
              label = labels,
            ),
            link = dict(
              source = [sources_map[sources[-1]], sources_map[lang], sources_map[lang], sources_map[lang], sources_map[lang], sources_map[lang], sources_map[lang], sources_map[lang], sources_map[lang]], # indices correspond to labels, eg A1, A2, A1, B1, ...
              target = [sources_map[lang], sources_map[sources[1]], sources_map[sources[2]], sources_map[sources[3]], sources_map[sources[4]], sources_map[sources[5]], sources_map[sources[6]], sources_map[sources[7]], sources_map[sources[8]]],
              value =  [porportion/all_proportion, annotation_count[sources[1]]/porportion, annotation_count[sources[2]]/porportion, annotation_count[sources[3]]/porportion, annotation_count[sources[4]]/porportion, annotation_count[sources[5]]/porportion, annotation_count[sources[6]]/porportion, annotation_count[sources[7]]/porportion, annotation_count[sources[8]]/porportion]
          ))])
        fig.update_layout(title_text=f"{self.lang_map[lang]} keyword annotation proportion", font_size=12)
        fig.show()
        return fig


# Heatmap

In [127]:
class overlap_heatmap:
    
    def __init__(self, version):
        self.version = version
        self.langs = ['ar', 'en', 'fr', 'es']
        self.lang_map = {'ar': 'Arabic', 'en': 'English', 'es': 'Spanish', 'fr': 'French'}
        self.language_data = pd.read_csv(f'{version}/keyword_annotations/store-all-messages.csv')
        
    def annotate(self, version, folder):
        langs_analysis = []
        for lang in self.langs:
            annotated_data = pd.read_csv(f'{version}/keyword_annotations/store-all-messages-annotated-v1-{lang}.csv')
            self.themes = [t for t in list(annotated_data.columns) if t !='M52/language']
            self.themes_mapping = dict(zip(list(self.themes),['politics and society', 'technology', 'covid-19', 'geopolotics', 'environment', 'military and security', 'economy', 'chinese culture and people']))
            annotated_renamed = self.rename_columns(annotated_data).drop('M52/language', axis = 1)
            self.build_heatmap(version, folder, df, lang)
        
            
    def analyze_overlap(self, df):
       
        return (df.T @ df) + ((0-df.T)@(0-df))
        

    def rename_columns(self, df):
        renamed = df.rename(columns = self.themes_mapping)
        return renamed

    def build_heatmap(self, folder, df, lang):
        fig = df.style.background_gradient(cmap ='viridis')\
            .set_properties(**{'font-size': '20px'})
        fig.set_caption(f'{self.lang_map[lang]} overlap heatmap').export_png(f'{self.version}/{folder}/{self.lang_map[lang]}_heatmap.png')
        return fig


# Project level

In [128]:
class project_level:
    
    def __init__(self):
        self.dc = accounts_split('v1')
        self.ka = keyword_annotator_split('v1')
        self.hm = overlap_heatmap('v1')
        
        
    def save(self, folder):
        self.dc.annotate(folder)
        self.ka.annotate(folder)
    def heatmap(self, folder):
        self.hm.annotate(folder)
        
        

In [129]:
pl = project_level()
pl.save('sankey_diagrams')
pl.heatmap('heatmap')

[14994, 145052, 10750, 13356, 184152]


TypeError: annotate() missing 1 required positional argument: 'folder'

In [ ]:
# import plotly.graph_objects as go
# import urllib, json
# import certifi

# url = 'https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json'
# response = urllib.request.urlopen(url, cafile=certifi.where())
# data = json.loads(response.read())

# # override gray link colors with 'source' colors
# opacity = 0.4
# # change 'magenta' to its 'rgba' value to add opacity
# data['data'][0]['node']['color'] = ['rgba(255,0,255, 0.8)' if color == "magenta" else color for color in data['data'][0]['node']['color']]
# data['data'][0]['link']['color'] = [data['data'][0]['node']['color'][src].replace("0.8", str(opacity))
#                                     for src in data['data'][0]['link']['source']]

# fig = go.Figure(data=[go.Sankey(
#     valueformat = ".0f",
#     valuesuffix = "TWh",
#     # Define nodes
#     node = dict(
#       pad = 15,
#       thickness = 15,
#       line = dict(color = "black", width = 0.5),
#       label =  data['data'][0]['node']['label'],
#       color =  data['data'][0]['node']['color']
#     ),
#     # Add links
#     link = dict(
#       source =  data['data'][0]['link']['source'],
#       target =  data['data'][0]['link']['target'],
#       value =  data['data'][0]['link']['value'],
#       label =  data['data'][0]['link']['label'],
#       color =  data['data'][0]['link']['color']
# ))])

# fig.update_layout(title_text="Energy forecast for 2050<br>Source: Department of Energy & Climate Change, Tom Counsell via <a href='https://bost.ocks.org/mike/sankey/'>Mike Bostock</a>",
#                   font_size=10)
# fig.show()

In [ ]:

fig = go.Figure(go.Sankey(
    arrangement = "snap",
    node = {
        "label": ["A", "B", "C", "D", "E", "F"],
        "x": [0.2, 0.1, 0.5, 0.7, 0.3, 0.5],
        "y": [0.7, 0.5, 0.2, 0.4, 0.2, 0.3],
        'pad':10},  # 10 Pixels
    link = {
        "source": [0, 0, 1, 2, 5, 4, 3, 5],
        "target": [5, 3, 4, 3, 0, 2, 2, 3],
        "value": [1, 2, 1, 1, 1, 1, 1, 2]}))

fig.show()

In [ ]:
fig

In [ ]:

# def language_summary(language_data):
#     langs_proportions = {}
#     for lang in langs:
#         langs_proportions[lang] = len(language_data[language_data['M52/language'] == lang])
#     langs_proportions['all'] = len(language_data)
#     return langs_proportions
        

# def rename_columns(df):
#     renamed = df.rename(columns=themes_mapping)
#     return renamed

# def annotation_count(df):
#     theme_count = {}
#     for t in list(themes_mapping.values()):
#         theme_count[t] = df[t].value_counts()[1]
#     return theme_count

# def create_dict(lst):
#     res_dct = {}
#     for i,t in enumerate(lst):
#         res_dct[t] = i
#     return res_dct

# def source_prefixes(all_proportion, proportion, sources, annotation_count):
#     annotated = []
#     annotated.append(f'{sources[0]} - {proportion}')
#     for i, ant in enumerate(sources[1:-1]):
#         annotated.append(f'{ant} - {annotation_count[sources[i+1]]}')
#     return annotated + [f'{sources[-1:][0]} - {all_proportion}']

# def keyword_annotation(all_proportion, lang, porportion, annotation_count):
#     sources = [ lang, "economy", "politics and society", "covid-19", "technology", "environment", "military and security", "geopolotics", "chinese culture and people", 'source data']    
#     sources_map = create_dict(sources)
#     labels = source_prefixes(all_proportion,  porportion, sources, annotation_count)
#     fig = go.Figure(data=[go.Sankey(
#         arrangement = "snap",
#          valueformat = ".0f",
#         node = dict(
#           pad = 15,
#           thickness = 20,
#           line = dict(color = "black", width = 0.5),
#           label = labels,
#         ),
#         link = dict(
#           source = [sources_map[sources[-1]], sources_map[lang], sources_map[lang], sources_map[lang], sources_map[lang], sources_map[lang], sources_map[lang], sources_map[lang], sources_map[lang]], # indices correspond to labels, eg A1, A2, A1, B1, ...
#           target = [sources_map[lang], sources_map[sources[1]], sources_map[sources[2]], sources_map[sources[3]], sources_map[sources[4]], sources_map[sources[5]], sources_map[sources[6]], sources_map[sources[7]], sources_map[sources[8]]],
#           value =  [porportion/all_proportion, annotation_count[sources[1]]/porportion, annotation_count[sources[2]]/porportion, annotation_count[sources[3]]/porportion, annotation_count[sources[4]]/porportion, annotation_count[sources[5]]/porportion, annotation_count[sources[6]]/porportion, annotation_count[sources[7]]/porportion, annotation_count[sources[8]]/porportion]
#       ))])
#     lang_map = {'ar': 'Arabic', 'en': 'English', 'es': 'Spanish', 'fr': 'French'}
#     fig.update_layout(title_text=f"{lang_map[lang]} keyword annotation proportion", font_size=12)
#     fig.show()